In [1]:
import logging
import argparse

import tensorflow as tf
from tensorflow.python.saved_model import utils as smutils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
from onnx_tf.backend import prepare
import onnx
import torch
import torchvision

## model load
import json
import pandas as pd
import os
import random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.utils.data import *
import easydict
import sys
sys.path.append('./Whatiswrong')
sys.path.append('./Scatter')
sys.path.append('./RobustScanner')

import BaseModel
import utils
from albumentations import GaussNoise, IAAAdditiveGaussianNoise, Compose, OneOf
from albumentations.pytorch import ToTensor

# opt
opt = easydict.EasyDict({
    "experiment_name" : f'{utils.SaveDir_maker(base_model = "www_jamo_vertical", base_model_dir = "./models")}',
    'saved_model' : 'RobustScanner_0908/2/best_accuracy_86.6.pth',
    "manualSeed" : 1111,
    "imgH" : 64 ,
    "imgW" :  256,
#     "imgW" :  256,
    "PAD" : True ,
    'batch_size' : 10,
    'data_filtering_off' : True,
    'workers' : 20,
    'rgb' :True,
    'sensitive' : True,
    'top_char' : ' !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅃㅆㅉ',
    'mid_char' : ' ㅏㅑㅓㅕㅗㅛㅜㅠㅡㅣㅐㅒㅔㅖㅘㅙㅚㅝㅞㅟㅢ',
    'bot_char' : ' ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅃㅆㅉㄳㄵㄶㄺㄻㄼㄽㄾㄿㅀㅄ',
    'batch_max_length' : 25,
    'num_fiducial' : 20,
    'output_channel' : 512,
    'hidden_size' :256,
    'lr' : 1,
    'rho' : 0.95,
    'eps' : 1e-8,
    'grad_clip' : 5,
    'valInterval' : 200,
    'num_epoch' : 100,
    'input_channel' : 3,
    'FT' : True,
    'extract' : 'resnet',
#     'extract' : 'RCNN',
    'pred' : ''
    })

device = torch.device('cpu')
top_converter = utils.AttnLabelConverter(opt.top_char, device)
middle_converter = utils.AttnLabelConverter(opt.mid_char, device)
bottom_converter = utils.AttnLabelConverter(opt.bot_char, device)
opt.top_n_cls = len(top_converter.character)
opt.mid_n_cls = len(middle_converter.character)
opt.bot_n_cls = len(bottom_converter.character)

attn_converter_top = utils.AttnLabelConverter(opt.top_char, device)
attn_converter_mid = utils.AttnLabelConverter(opt.mid_char, device)
attn_converter_bot = utils.AttnLabelConverter(opt.bot_char, device)
opt.top_n_cls_attn = attn_converter_top.n_cls
opt.mid_n_cls_attn = attn_converter_mid.n_cls
opt.bot_n_cls_attn = attn_converter_bot.n_cls

In [2]:
# log = logging.getLogger(__name__)
# logging.basicConfig(level=logging.INFO)

In [3]:
model = BaseModel.model(opt, device)
model.load_state_dict(torch.load('./models/RobustScanner_0912/0/best_accuracy_90.36.pth', map_location='cpu')) 
model.to(device)
_ = model.eval()

In [8]:
def export_onnx(model, dummy_input, file, input_names, output_names,
                num_inputs):
    """
    Converts a Pytorch model to the ONNX format and saves the .onnx model file.
    The first dimension of the input nodes are of size N, where N is the
    minibatch size. This dimensions is here replaced by an arbitrary string
    which the ONNX -> TF library interprets as the '?' dimension in Tensorflow.
    This process is applied because the input minibatch size should be of an
    arbitrary size.
    :param model: Pytorch model instance with loaded weights
    :param dummy_input: tuple, dummy input numpy arrays that the model
        accepts in the inference time. E.g. for the Text+Image model, the
        tuple would be (np.float32 array of N x W x H x 3, np.int64 array of
        N x VocabDim). Actual numpy arrays values don't matter, only the shape
        and the type must match the model input shape and type. N represents
        the minibatch size and can be any positive integer. True batch size
        is later handled when exporting the model from the ONNX to TF format.
    :param file: string, Path to the exported .onnx model file
    :param input_names: list of strings, Names assigned to the input nodes
    :param output_names: list of strings, Names assigned to the output nodes
    :param num_inputs: int, Number of model inputs (e.g. 2 for Text and Image)
    """
    # List of onnx.export function arguments:
    # https://github.com/pytorch/pytorch/blob/master/torch/onnx/utils.py
    # ISSUE: https://github.com/pytorch/pytorch/issues/14698
    torch.onnx.export(model, args=dummy_input, input_names=input_names, opset_version=10,
                      output_names=output_names, f=file)

    # Reload model to fix the batch size
    model = onnx.load(file)
    model = make_variable_batch_size(num_inputs, model)
    onnx.save(model, file)

    log.info("Exported ONNX model to {}".format(file))


In [9]:
home = '/home'
base_path = home+'/FoodDetection/AI_OCR/val'
# base_path = home+'/Data/KoreanSTR/Combined_OCR_Data/For_Recognition'
files = os.listdir(base_path)
files.remove('.ipynb_checkpoints')
val_data =[]
for file in files:
    val_data.append([os.path.join(base_path, file), ' ', ' ', ' '])

stream = utils.CustomDataset_jamo(val_data, resize_shape= (opt.imgH, opt.imgW), transformer=ToTensor())
loader = DataLoader(stream, batch_size = 61, shuffle=False, num_workers=1 )
iterer = iter(loader)

# img, label = next(iterer)
img, label_top, label_mid, label_bot = next(iterer)

text_top, length_top = attn_converter_top.encode(label_top, opt.batch_max_length)
text_mid, length_mid = attn_converter_mid.encode(label_mid, opt.batch_max_length)
text_bot, length_bot = attn_converter_bot.encode(label_bot, opt.batch_max_length)
batch_size = img.size(0)
file = 'torch2tf.onnx'
input_names = ['cropped_img']
output_names = ['logits']

In [10]:
export_onnx(model, (img, text_bot[:, :-1], text_mid[:, :-1], text_top[:, :-1]) , file, 
            input_names, output_names, len(img))

/usr/local/lib/python3.6/dist-packages/torch/onnx/utils.py:738: UserWarning: ONNX export failed on ATen operator grid_sampler because torch.onnx.symbolic_opset10.grid_sampler does not exist
  .format(op_name, opset_version, op_name))


RuntimeError: Exporting the operator grid_sampler to ONNX opset version 10 is not supported. Please open a bug to request ONNX export support for the missing operator.

# onnx doesn't not support 'grid_sampler' layer so far